In [1]:
import gym

In [2]:
env = gym.make('SpaceInvaders-v0')

### Random Agent

In [3]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        state, reward, done, info=env.step(env.action_space.sample())
        score += reward
    print('Episode : {}\nScore: {}\nInfo: {}'.format(episode,score,info))
    
env.close()

Episode : 1
Score: 135.0
Info: {'ale.lives': 0}
Episode : 2
Score: 105.0
Info: {'ale.lives': 0}
Episode : 3
Score: 130.0
Info: {'ale.lives': 0}
Episode : 4
Score: 135.0
Info: {'ale.lives': 0}
Episode : 5
Score: 45.0
Info: {'ale.lives': 0}
Episode : 6
Score: 20.0
Info: {'ale.lives': 0}
Episode : 7
Score: 45.0
Info: {'ale.lives': 0}
Episode : 8
Score: 65.0
Info: {'ale.lives': 0}
Episode : 9
Score: 185.0
Info: {'ale.lives': 0}


### DQNA

In [4]:
#### Import NN Packs
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [29]:
def Bmodel(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
    

In [30]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [31]:
model = Bmodel(height, width, channels, actions)

In [32]:
#### Importing Keras-rl2

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [33]:
def Bagent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2 ,nb_steps=9999)
    memory = SequentialMemory(limit=99999, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions = actions, enable_dueling_network=True, 
                   nb_steps_warmup=9999)
    return dqn

In [34]:
dqn = Bagent(model, actions)

In [35]:
dqn.compile(Adam(learning_rate=0.00001))

In [36]:
dqn.fit(env, nb_steps=99999, visualize=False ,verbose=1)

Training for 99999 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:26 - reward: 0.0000e+00

c:\users\ghaiyur\documents\rl-agents\sirl\env\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 125s 12ms/step - reward: 0.1910
15 episodes - episode_reward: 124.000 [55.000, 210.000] - ale.lives: 2.150

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 6473s 647ms/step - reward: 0.1840
15 episodes - episode_reward: 119.333 [10.000, 230.000] - loss: 0.989 - mean_q: 3.072 - mean_eps: 0.100 - ale.lives: 1.930

Interval 3 (20000 steps performed)
   44/10000 [..............................] - ETA: 1:45:55 - reward: 0.3409done, took 6626.367 seconds


In [37]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 130.000, steps: 1011
Episode 2: reward: 115.000, steps: 764
Episode 3: reward: 305.000, steps: 935
Episode 4: reward: 35.000, steps: 495
Episode 5: reward: 60.000, steps: 682
Episode 6: reward: 225.000, steps: 1298
Episode 7: reward: 395.000, steps: 1443
Episode 8: reward: 25.000, steps: 412
Episode 9: reward: 420.000, steps: 1128
Episode 10: reward: 90.000, steps: 698
Episode 11: reward: 55.000, steps: 1054
Episode 12: reward: 45.000, steps: 608
Episode 13: reward: 15.000, steps: 570
Episode 14: reward: 180.000, steps: 844
Episode 15: reward: 240.000, steps: 1064
Episode 16: reward: 350.000, steps: 995
Episode 17: reward: 235.000, steps: 1157
Episode 18: reward: 65.000, steps: 571
Episode 19: reward: 200.000, steps: 960
Episode 20: reward: 450.000, steps: 1659
Episode 21: reward: 245.000, steps: 883
Episode 22: reward: 235.000, steps: 1006
Episode 23: reward: 40.000, steps: 683
Episode 24: reward: 215.000, steps: 805
Episode 25: reward: 3

In [38]:
dqn.save_weights('models/dqn.h5f')

In [ ]:
# dqn.load_weights('models/dqn.h5f')